In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import datetime
from sklearn.model_selection import GridSearchCV

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [2]:
df_train_xgboost = df_train
df_test_xgboost = df_test

def asignarMetros(metroscubiertos, metrostotales):
    if (metroscubiertos != 0 and metrostotales == 0):
        return metroscubiertos
    else:
        return metrostotales
    
def asignarMetrosLibres(metroscubiertos, metrostotales):
    if (metroscubiertos != metrostotales):
        return metrostotales-metroscubiertos
    else:
        return 0
    
#df_train_xgboost['metrostotales'].fillna(0, inplace=True)
#df_train_xgboost['metrostotales'] = df_train_xgboost.apply(lambda x: asignarMetros(x['metroscubiertos'],x['metrostotales']),axis=1)
#df_train_xgboost['metroslibres'] = df_train_xgboost.apply(lambda x: asignarMetrosLibres(x['metroscubiertos'],x['metrostotales']),axis=1)

#df_test_xgboost['metrostotales'].fillna(0, inplace=True)
#df_test_xgboost['metrostotales'] = df_test_xgboost.apply(lambda x: asignarMetros(x['metroscubiertos'],x['metrostotales']),axis=1)
#df_test_xgboost['metroslibres'] = df_test_xgboost.apply(lambda x: asignarMetrosLibres(x['metroscubiertos'],x['metrostotales']),axis=1)

In [3]:
#----------------PREPROCESAMIENTO----------------

df_train_xgboost = df_train_xgboost.drop(['ciudad','fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
df_test_xgboost = df_test_xgboost.drop(['ciudad','fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
#df_train_xgboost = df_train_xgboost.fillna(value = {'provincia': 0, 'tipodepropiedad': 0})

print(df_train_xgboost.shape)
#XGBoost maneja los NaNs, ver despues si lo dejamos como esta

df_train_xgboost = pd.get_dummies(df_train_xgboost, drop_first=True)#, sparse = True)
df_test_xgboost = pd.get_dummies(df_test_xgboost, drop_first=True)#, sparse = True)

print(df_train_xgboost.shape)
precio = df_train_xgboost['precio']
df_train_xgboost.drop(labels=['precio'], axis=1,inplace = True)
df_train_xgboost.insert(len(df_train_xgboost.columns), 'Precio', precio)

X_train, y_train = df_train_xgboost.iloc[:,:-1],df_train_xgboost.iloc[:,-1]

df_test_xgboost['tipodepropiedad_Garage'] = 0
df_test_xgboost['tipodepropiedad_Hospedaje'] = 0
df_test_xgboost = df_test_xgboost[list(X_train.columns.values)]

xgb1 = XGBRegressor()

#Ir cambiando parametros para ver como functiona

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.04], #probamos con 0.03,0.04,0.05,0.06,0.07
              'max_depth': [9], #probamos con 5,6,7,8,9,10,11
              'min_child_weight': [1], #Probamos con 1,2,3,4,5,6
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7], #probamos 0.3,0.4,0.5,0.6,0.7,0.8
              'n_estimators': [700],  #Se probo con 400,500,600,700,800
            'gamma': [0]} #Se probo con 0,1,5,10

#Estos son sin correccion de metros
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7214816192533363
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7218556040564428
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7219040830119198
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 9, 'min_child_weight': 4, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7242695855881088
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 9, 'min_child_weight': 2, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7245128843441556
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7246402143520269


xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,
         y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

(240000, 15)
(240000, 67)
Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 out of   8 | elapsed: 15.8min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   8 out of   8 | elapsed: 15.8min finished
c:\users\manuel\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
c:\users\manuel\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


0.7246402143520269
{'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.04, 'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
